In [8]:
#using tutorial at: https://www.nbinteract.com/tutorial/tutorial_interact.html

# Using Interact

The ipywidgets library provides the simplest way to get started writing interactive documents. Although the library itself has its own documentation, we will provide a quick overview to let you get started as quickly as possible.

We start by importing the interact function:

In [1]:
from ipywidgets import interact

The interact function takes in a function and produces an interface to call the function with different parameters.

In [2]:
def square(x):
    return x * x

Pass the square function into interact and specify square's arguments as keyword arguments:

In [3]:
interact(square, x=10)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.square(x)>

To control the range of values x can take, you can pass in a tuple of the same format as Python's range function:

In [4]:
interact(square, x=(0,100,10));

interactive(children=(IntSlider(value=50, description='x', step=10), Output()), _dom_classes=('widget-interact…

Notice how dragging the slider changes the input to the square function and automatically updates the output. This is a powerful idea that we will build on for the rest of this tutorial.

By using interact, you can build complex widget-based interfaces in a notebook. These widgets will appear in your resulting webpage, giving your page interactivity.

# Widget Types

Notice that interact automatically generates a slider because the argument is numeric. Other types of arguments will generate different types of interfaces. For example, a string will generate a textbox.

In [5]:
def friends(name, number):
    return '{} has {} friends!'.format(name, number)

In [6]:
interact(friends,name='Sam', number=(5,10));

interactive(children=(Text(value='Sam', description='name'), IntSlider(value=7, description='number', max=10, …

And a dictionary will generate a dropdown menu:

In [7]:
interact(friends, name='Sam', number={'One': 1, 'Five':5, 'Ten':10});

interactive(children=(Text(value='Sam', description='name'), Dropdown(description='number', options={'One': 1,…

# Publishing A Webpage

To convert a notebook into an HTML file, start a terminal and run the following command.